# Average 3 Fold predictions

In [19]:
import json
import numpy as np
from pathlib import Path
import pandas as pd

# fold_paths = [
#     "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-H/Best-Setting/Fold-1",
#     "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-H/Best-Setting/Fold-2",
#     "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-H/Best-Setting/Fold-3"
# ]

fold_paths = [
    "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViTStarDist/Common-Loss/ViT256/Unshared-decoder/CPP-Net-Setting/Fold-1",
    "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViTStarDist/Common-Loss/ViT256/Unshared-decoder/CPP-Net-Setting/Fold-2",
    "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViTStarDist/Common-Loss/ViT256/Unshared-decoder/CPP-Net-Setting/Fold-3"
]


# fold_paths = [
#     "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-L/Best-Setting/Fold-1",
#     "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-L/Best-Setting/Fold-2",
#     "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-L/Best-Setting/Fold-3"
# ]

# fold_paths = [
#     #"/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-L/Best-Setting/2023-09-19T123306_CellViT-SAM-L-Fold-1",
#     #"/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-L/Best-Setting/2023-09-19T123306_CellViT-SAM-L-Fold-2",
#     # "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-L/Best-Setting/2023-09-19T221050_CellViT-SAM-L-Fold-3"
# ]

In [20]:
detection_results = []
cell_pq_results = []
tissue_metrics = []
global_f1 = []
global_prec = []
global_rec = []


for fold, fold_path in enumerate(fold_paths):
    json_path = Path(fold_path) / "inference_results.json"
    
    with open(json_path, 'r') as file:
        data = json.load(file)
    detection_results.append(pd.DataFrame.from_dict(data["nuclei_metrics_d"]))
    cell_pq_results.append(pd.DataFrame(data["nuclei_metrics_pq"], index=[0]))
    tissue_metrics.append(pd.DataFrame.from_dict(data["tissue_metrics"]))
    global_f1.append(float(data["dataset"]["f1_detection"]))
    global_prec.append(data["dataset"]["precision_detection"])
    global_rec.append(data["dataset"]["recall_detection"])

detection_avg = pd.concat([each.stack() for each in detection_results],axis=1)\
             .apply(lambda x:x.mean(),axis=1)\
             .unstack()
cell_pq_avg = pd.concat([each.stack() for each in cell_pq_results],axis=1)\
             .apply(lambda x:x.mean(),axis=1)\
             .unstack()           
tissue_metrics_avg = pd.concat([each.stack() for each in tissue_metrics],axis=1)\
             .apply(lambda x:x.mean(),axis=1)\
             .unstack()
             
global_f1 = np.mean(global_f1)
global_prec = np.mean(global_prec)
global_rec = np.mean(global_rec)

### Detection-Scores

In [21]:
detection_avg.T

,f1_cell,prec_cell,rec_cell
Neoplastic,0.599791,0.608217,0.591758
Inflammatory,0.533746,0.560678,0.509443
Connective,0.436772,0.444615,0.429600
Dead,0.249274,0.225969,0.278972
Epithelial,0.576339,0.600444,0.555460


In [22]:
print(f"f1: {global_f1}")
print(f"prec: {global_prec}")
print(f"rec: {global_rec}")

f1: 0.7619087686580319
prec: 0.777588247819053
rec: 0.7472319754008732


### Cell-Type-PQ-Scores

In [23]:
cell_pq_avg

,Neoplastic,Inflammatory,Connective,Dead,Epithelial
0,0.423096,0.349143,0.240301,0.081575,0.38168


### Tissue-Type-PQ-Scores

In [6]:
tissue_metrics_avg.T

,Dice,Jaccard,mPQ,bPQ
adrenal_gland,0.832176,0.727592,0.430762,0.660475
bile-duct,0.784503,0.706498,0.436279,0.632253
bladder,0.792025,0.719128,0.554560,0.660057
breast,0.826190,0.717780,0.468221,0.628305
cervix,0.723431,0.689100,0.461308,0.644862
colon,0.708932,0.652689,0.377279,0.518341
esophagus,0.824041,0.721220,0.483496,0.607718
headneck,0.673978,0.642576,0.435510,0.596915
kidney,0.788237,0.693075,0.474722,0.658667
liver,0.856573,0.757436,0.460317,0.685766


In [7]:
row_mean = tissue_metrics_avg.mean(axis=1)
row_mean

Dice       0.800534
Jaccard    0.709582
mPQ        0.441663
bPQ        0.626377
dtype: float64

In [8]:
row_mean = tissue_metrics_avg.std(axis=1)
row_mean

Dice       0.051455
Jaccard    0.033341
mPQ        0.049993
bPQ        0.039872
dtype: float64